In [ ]:
# DO NOT modify this cell.
filename = "Buchtel.pgm"
verticalSeam2Remove = 10
horizontalSeam2Remove = 8

# Yes, you can create your own test cases and you should. Do it in a new cell.

In [7]:
import numpy as np

def load_pgm_as_numpy_array(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Check if the file is in PGM format
    if lines[0].strip() != 'P2':
        raise ValueError("File is not in PGM format (P2)")

    # Extract image dimensions and maximum pixel value
    width, height = map(int, lines[2].split())
    max_value = int(lines[3])

    # Extract pixel values from the remaining lines
    pixel_values = []
    for line in lines[4:]:
        pixel_values.extend(map(int, line.split()))

    # Convert pixel values to a numpy array
    image_np = np.array(pixel_values).reshape(height, width)

    return image_np, max_value

def calculate_energy(image):
    """Calculate energy of each pixel in the image with edge handling."""
    rows, cols = image.shape
    energy = np.zeros_like(image)

    for y in range(0, rows):
        for x in range(0, cols):
            energy[y][x] = abs(image[y,x] - image[y-1 if y-1>=0 else 0,x]) + \
                           abs(image[y,x] - image[y+1 if y+1<rows else rows-1,x]) + \
                           abs(image[y,x] - image[y,x-1 if x-1>=0 else 0]) + \
                           abs(image[y,x] - image[y,x+1 if x+1<cols else cols-1])
    return energy

def calculate_cumulative_energy(energy):
    rows, cols = energy.shape
    cumulative_energy = np.zeros_like(energy)

    # Copy the first row of energy into cumulative_energy
    cumulative_energy[0, :] = energy[0, :]

    # Fill in the rest of the cumulative energy matrix
    for y in range(1, rows): # Start from the second row
        for x in range(0, cols):
            cumulative_energy[y][x] = min(energy[y][x] + cumulative_energy[y-1][x-1] if x-1>=0 else float('inf'),
                                          energy[y][x] + cumulative_energy[y-1][x],
                                          energy[y][x] + cumulative_energy[y-1][x+1] if x+1<cols else float('inf'))

    return cumulative_energy

def seam_carve(image):
    """Perform seam carving on the image."""
    energy = calculate_energy(image)
    cumulative_energy = calculate_cumulative_energy(energy)

    rows, cols = image.shape

    # Find the minimum value in the last row of cumulative_energy
    min_value_x = np.argmin(cumulative_energy[-1, :])

    # Initialize seam path to store the indices of pixels to be removed
    seam_path = []

    # Trace back to find the seam path
    for y in range(rows-1, -1, -1):
        seam_path.append((y, min_value_x))

        # Determine the next column index for the seam path
        if y >= 0:

            # x indices of the three possible pixels in the row above
            up_left_x = min_value_x - 1
            up_x = min_value_x
            up_right_x = min_value_x + 1

            # values of the three possible pixels in the row above
            up_left = cumulative_energy[y-1][up_left_x] if up_left_x >= 0 else float('inf')
            up = cumulative_energy[y-1][up_x]
            up_right = cumulative_energy[y-1][up_right_x] if up_right_x < cols else float ('inf')

            # get the min value
            min_value = min(up_left, up, up_right)

            # piroritize up_left first
            if(min_value == up_left):min_value_x = up_left_x
            elif(min_value == up):min_value_x = up_x
            elif(min_value == up_right):min_value_x = up_right_x
            else: raise Exception("No minium value found in row above")

    # Create a copy of the image without the pixels in the seam path
    carved_image = np.zeros((rows, cols - 1), dtype=image.dtype)
    for y, x in seam_path:
        # Exclude the pixel at (y, x) from the image
        carved_image[y, :] = np.delete(image[y, :], x, axis=0)

    return carved_image

image = np.array([
    [2, 10, 70],
    [155, 30, 10],
    [200, 190, 120],
    [10, 255, 60]
])

# Perform seam carving on the image
carved_image = seam_carve(image)

# Display the resulting carved image
print("Carved Image:")
print(carved_image)

Carved Image:
[[  2  70]
 [155  30]
 [200 190]
 [ 10 255]]


In [2]:
import os

def save_pgm(image_filepath, image_np, max_value, v, h):
    """Save a numpy array as a PGM file."""
    base_filename, ext = os.path.splitext(os.path.basename(image_filepath))
    output_directory = 'test/myProcessed/'
    output_filename = f"{output_directory}{base_filename}_processed_{v}_{h}.pgm"

    with open(output_filename, 'w') as file:
        # Write PGM header
        file.write("P2\n")
        file.write(f"# Created by IrfanView\n")
        file.write(f"{image_np.shape[1]} {image_np.shape[0]}\n")  # Width and height
        file.write(f"{max_value}\n")  # Maximum pixel value

        # Write pixel values
        for row in image_np:
            file.write(" ".join(map(str, row)) + "\n")

    print(f"Processed image saved as: {output_filename}")

def process_image(image_filepath, num_seams):
    """Process the image by seam carving and save the result."""
    image_np, max_value = load_pgm_as_numpy_array(image_filepath)
    # Perform seam carving 'num_seams' times
    for i in range(num_seams):
        image_np = seam_carve(image_np)  # Seam carve the image
    save_pgm(image_filepath, image_np, max_value, num_seams, 0)  # Save the processed image

# User inputs
# image_filename = input("Enter the image file name: ")
# num_seams = int(input("Enter the number of vertical seams to remove: "))

# For testing
image_filepath = './test/raw/CAS.pgm'
vertical_seams = 20

# Entry Point for Part I
process_image(image_filepath, vertical_seams)


Processed image saved as: test/myProcessed/CAS_processed_20_0.pgm


In [13]:
# Test cell

my = "./test/myProcessed/CAS_processed_20_0.pgm"
correct = "./test/processed/CAS_processed_20_0.pgm"

import filecmp
same = filecmp.cmp(my, correct)
print(same, same)

False False


In [ ]:
# Part II : both vertical and horizontal seams removal
# Save your processed file to img_processed_v_h.pgm
filename2 = filename.split(".")[0]+"_processed_"+ \
            str(verticalSeam2Remove)+"_"+str(horizontalSeam2Remove)+".pgm"
print(filename2)

# your code, add cells as you need

In [ ]:
# This is for graduate students only
filenameG = "flower.ppm"
vSeam2Remove = 20
hSeam2Remove = 25

In [ ]:
# This is for graduate students only
# Part III : color img in ppm
filenameG2 = filenameG.split(".")[0]+"_processed_"+ \
            str(vSeam2Remove)+"_"+str(hSeam2Remove)+".ppm"
print(filenameG2)
# your code, add cells as you need